In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
if torch.cuda.is_available():
    print(f"Running on GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Not running on GPU")


Not running on GPU


In [4]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
len(words)

32033

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i,s in enumerate(chars)}
stoi['.'] = 0 # character key, integer value (dictionary)
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [62]:
# 
block_size = 3
X, Y = [], []
for w in words[:5]:

    # print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [61]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([36, 4]), torch.int64, torch.Size([36]), torch.int64)

In [9]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [10]:
Y

tensor([ 5, 13, 13,  ..., 26, 24,  0])

In [11]:
C = torch.randn((27,2))
# lookup table 

In [12]:
C[5]

tensor([-0.1629, -0.3168])

In [13]:
# F.one_hot(torch.tensor(5), num_classes = 27).float()

In [14]:
# F.one_hot(torch.tensor(5), num_classes = 27).float() @ C

In [15]:
C[X].shape

torch.Size([228146, 3, 2])

In [16]:
X[13,2]

tensor(1)

In [17]:
C[X][13,2]

tensor([ 0.2521, -0.8766])

In [18]:
C[1]

tensor([ 0.2521, -0.8766])

In [19]:
emb = C[X]
emb.shape
# input

torch.Size([228146, 3, 2])

In [20]:
W1 = torch.randn((6,100))
# 6 weights (3 inputs * 2 dimensionality), 100 neurons
b1 = torch.randn(100)

In [21]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [22]:
h

tensor([[ 0.9608,  0.7078, -0.9962,  ..., -0.9962, -0.0133,  0.9983],
        [ 0.9333, -0.7921, -0.9561,  ..., -0.9749,  0.9545,  0.9908],
        [ 0.9822,  0.5886,  0.1895,  ..., -0.8567, -0.5049,  0.9976],
        ...,
        [ 0.6152, -0.5526,  0.3101,  ..., -0.3801,  0.5794,  0.9373],
        [ 0.8535,  0.8032,  0.7354,  ..., -0.2699, -0.8213,  0.9786],
        [-0.3883,  0.6498, -0.1658,  ..., -0.4332, -0.8702,  0.9310]])

In [23]:
h.shape

torch.Size([228146, 100])

In [24]:
torch.cat([emb[:, 0,:], emb[:, 1,:], emb[:, 2,:]], 1).shape

torch.Size([228146, 6])

In [25]:
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([228146, 6])

In [26]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [27]:
a.shape

torch.Size([18])

In [28]:
a.view(3,2,3)

tensor([[[ 0,  1,  2],
         [ 3,  4,  5]],

        [[ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17]]])

In [29]:
# a.storage()

In [30]:
# emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

In [31]:
W2 = torch.randn((100,27))
# 100 inputs (past neurons) to 27 outputs (neurons for possible letter outputs)
b2 = torch.randn(27)

In [32]:
logits = h @ W2 + b2
# ^ outputs of neural net

In [33]:
logits.shape

torch.Size([228146, 27])

In [34]:
logits

tensor([[  4.4214,  -9.4018, -11.2197,  ...,  15.8643,  -5.7515, -10.5196],
        [ 16.8104,  -8.8690,  -3.7558,  ...,   9.9349, -10.2777,   0.1607],
        [ -0.2401,  -1.1969,  -3.3854,  ...,  -0.4258,  -5.1244,   6.4437],
        ...,
        [  3.0707,  -7.0193,   6.4316,  ...,  -3.0526,  -6.2131,   7.2560],
        [ -3.1457,  -3.6714,   4.8322,  ...,  -5.2289,  -3.8061,   5.9667],
        [ -6.7270,  -5.9049,  14.4003,  ...,  -0.7992,  -3.7090, -12.3647]])

In [35]:
count = logits.exp()

In [36]:
probs = count / count.sum(1, keepdims=True)

In [37]:
probs.shape

torch.Size([228146, 27])

In [38]:
probs[0].sum()

tensor(1.0000)

In [39]:
torch.arange(32)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [40]:
Y

tensor([ 5, 13, 13,  ..., 26, 24,  0])

In [41]:
# loss = -probs[torch.arange(32), Y].log().mean()
# loss
# indexes into torch and assigns probability for each Y
# take log, avg, and - to get neg loss likelihood

In [42]:
# ?>>>>>>>>>>>>>>>>>>>>>>>>>>>>> SUMMARY SO FAR:

In [43]:
def build_dataset(words):
    block_size = 3
    X, Y = [], []
    for w in words:
    
        # print(w)
        context = [0] * block_size 
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(.8*len(words))
n2 = int(.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [44]:
#dataset:
Xtr.shape, Ytr.shape

(torch.Size([182625, 3]), torch.Size([182625]))

In [45]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,10), generator = g)
W1 = torch.randn((30,200), generator = g)
b1 = torch.randn(200, generator = g)
# second layer - softmax
W2 = torch.randn((200,27), generator = g)
b2 = torch.randn(27, generator = g)
parameters = [C, W1, b1, W2, b2]

In [46]:
sum(p.nelement() for p in parameters)

11897

In [47]:
for p in parameters:
    p.requires_grad = True

In [48]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [49]:
lri = []
lossi = []
stepi = []

In [56]:
for i in range(1):

    #minibatch
    ix = torch.randint(0,Xtr.shape[0], (32,))
    # (32,) = size: 1D tensor with 32 randomly-selected indices
    print(ix)
    
    # forward pass
    print(Xtr)f
    print(Xtr.shape)
    emb = C[Xtr[ix]] #(32, 3, 2)
    h = torch.tanh(emb.view(-1,30) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2 #(32, 27)
    # count = logits.exp()
    # probs = count / count.sum(1, keepdims=True)
    # loss = -probs[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits, Ytr[ix])
    # print(loss.item())
    
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    # lr = lrs[i]
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    #track stats
    # lri.append(lre[i])
    stepi.append(i)
    # lossi.append(loss.item())
    lossi.append(loss.log10().item())

# print(loss.item())

tensor([139288,  87035, 136043, 135015,  75962, 107768,  47617,  25979, 122147,
        123765,  32144,  21357,  13020,  22591,   9479,  35447,  90210,  45700,
        122597, 180362, 155072,  81829, 100130, 152603,  17333, 112946, 175695,
         15209,  12380,  16879,  30914,  23507])
tensor([[ 0,  0,  0],
        [ 0,  0, 25],
        [ 0, 25, 21],
        ...,
        [15, 12,  4],
        [12,  4,  1],
        [ 4,  1, 14]])
torch.Size([182625, 3])


In [ ]:
# plt.plot(lri, lossi)
plt.plot(stepi, lossi)

torch.randint(0,X.shape[0], (32,))

In [ ]:
# training batch's loss:
emb = C[Xtr] #(32, 3, 2)
h = torch.tanh(emb.view(-1,40) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 #(32, 27)
loss = F.cross_entropy(logits, Ytr)
loss

In [ ]:
# validation loss (beat 2.12):
emb = C[Xdev] #(32, 3, 2)
h = torch.tanh(emb.view(-1,40) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 #(32, 27)
loss = F.cross_entropy(logits, Ydev)
loss

In [ ]:
emb = C[Xte] #(32, 3, 2)
h = torch.tanh(emb.view(-1,40) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 #(32, 27)
loss = F.cross_entropy(logits, Yte)
loss

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data, C[:,1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color="white")
plt.grid('minor')

In [ ]:
# training split (80%), dev/validation split (10%), test split (10%)

In [ ]:
# sample model:

g = torch.Generator().manual_seed(2147483647+10)
for _ in range(20):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 +b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples = 1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))